In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_vccmmpo08',
    proc_date,
    'temp_sap_vccmmpo08'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmpo08
( 
  hash_id BIGINT,
  recordmode STRING,
  rocancel STRING,
  ebeln STRING,
  ebelp INT,
  abeln STRING,
  abelp INT,
  aurel INT,
  proc_date DATE,
  file_creation_ts TIMESTAMP
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'true',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'true'
)
""")

In [0]:
spark.sql(
    """CREATE OR REPLACE TEMP VIEW sap_vccmpo08 AS
    WITH dat AS (
        SELECT  CASE WHEN recordmode ='' THEN NULL ELSE recordmode END AS recordmode,	
                CASE WHEN rocancel ='' THEN NULL ELSE rocancel END AS rocancel,
                CASE WHEN ebeln='' THEN NULL ELSE ebeln END AS ebeln,
                TRY_CAST(LTRIM('0',ebelp) AS INTEGER) AS ebelp,	
                CASE WHEN abeln='' THEN NULL ELSE LTRIM('0',abeln) END AS abeln,	
                TRY_CAST(LTRIM('0',abelp) AS INTEGER) AS abelp,	
                TRY_CAST(LTRIM('0',aurel) AS INTEGER) AS aurel,	
                proc_date,
                file_creation_ts
        FROM    (
                    SELECT * EXCEPT(rn) FROM (
                        SELECT *, ROW_NUMBER() OVER (PARTITION BY reqtsn, datapakid, record ORDER BY proc_date DESC) rn FROM temp_sap_vccmmpo08  
                    ) a WHERE rn = 1 
                ) a
    )
    SELECT 
        FARM_FINGERPRINT(CONCAT(
                coalesce(CAST(recordmode AS STRING), ""), 
                coalesce(CAST(rocancel AS STRING), ""),
                coalesce(CAST(ebeln AS STRING), ""),
                coalesce(CAST(ebelp AS STRING), ""),
                coalesce(CAST(abeln AS STRING), ""),
                coalesce(CAST(abelp AS STRING), ""),
                coalesce(CAST(aurel AS STRING), "")
            )) AS hash_id,
        *
    FROM dat
;
"""
)

In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmpo08 main_tbl
USING sap_vccmpo08 temp_tbl
ON main_tbl.hash_id = temp_tbl.hash_id
WHEN NOT MATCHED THEN INSERT *;
""")